# Open Power System Data: time series
## About this notebook
This is a python script that downloads and processes time-series data from European power systems. Data include electricity consumption (load) from ENTSO-E, wind and solar power generation from various transmission system operators, and wind and solar capacity data from different sources. Processing and data sources are documented in this notebook.

The script produces two CSV files containing all data: one on hourly resolution and one in quarter-hourly resolution. The latter containts only those parameters that are available in quarter-hourly resolution.

Download all data usually takes several hours. The two output files are about 50 MB each.

## Data sources
For an overview of power system data sources see our [Data Sources](http://open-power-system-data.org/data-sources) project page. 

We use **load **data from
- [ENTSO-E](https://www.entsoe.eu/data/data-portal/consumption/Pages/default.aspx)

We use **wind and solar generation** data from
- [50Hertz](http://www.50hertz.com/en/Grid-Data)
- [Amprion](http://www.amprion.net/en/grid-data)
- [TransnetBW](https://www.transnetbw.com/en/key-figures)
- [TenneT](http://www.tennettso.de/site/en/Transparency/publications/network-figures/overview)

We use **wind and solar capacity** data from
- ...

We use **spot price** data from
- ...

## Links to the other Notebooks

- [download](https://github.com/Open-Power-System-Data/datapackage_timeseries/blob/master/download.ipynb)
- [read](https://github.com/Open-Power-System-Data/datapackage_timeseries/blob/master/read.ipynb)
- [processing](https://github.com/Open-Power-System-Data/datapackage_timeseries/blob/master/processing.ipynb)